In [2]:
import json
import string
from multiprocessing import Pool, cpu_count

import lightfm
import numpy as np
import pandas as pd
#import pymorphy2
import scipy.sparse
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
user = pd.read_csv('./kaggle/user_features_full.csv')
user.head()

,user_id,orders,retailer,city_name,total,prefer_discounts,master_category_id,parent_category_id
0,83,1,METRO,Московская Область,170.171765,False,"[432, 432, 432, 432, 434, 435, 328, 343, 343, ...","[429, 429, 429, 429, 429, 429, 321, 290, 290, ..."
1,142,1,METRO,Московская Область,3344.000000,False,[581],[9]
2,187,1,ВкусВилл,Москва,277.110001,False,"[85, 91]","[84, 90]"
3,224,1,Ашан,Москва,636.272493,False,"[61, 61, 61, 598, 600, 600, 630, 40, 91, 91, 9...","[56, 56, 56, 51, 56, 56, 74, 35, 90, 90, 90, 90]"
4,260,2,METRO,Москва,220.464167,False,"[76, 76, 76, 76, 630, 630, 630, 630, 80, 110, ...","[74, 74, 74, 74, 74, 74, 74, 74, 74, 99, 99, 9..."


In [4]:
items = pd.read_csv("./kaggle/products_lexical.csv")
items = items.set_index("itemId")
items.head()

,product_id,label,description
0,6,Питьевой йогурт Активиа чернослив 2% 290 г бзмж,Активиа 2 % Обезжиренное молоко Обезжиренное м...
1,8,"Питьевой йогурт Активиа натуральный 2,4% 290 г...","Активиа 2,4 % Обезжиренное молоко Обезжиренное..."
2,9,"Питьевой йогурт Активиа злаки 2,2% 290 г бзмж","Активиа 2,2 % Обезжиренное молоко Обезжиренное..."
3,10,"Питьевой йогурт Активиа злаки-яблоко 2,2% 290 ...","Активиа 2,2 г 13,2 г Продукт во вскрытой упако..."
4,11,"Питьевой йогурт Активиа натуральный 2,4% 870 г...","Активиа Коровье Бифидобактерии 3,2 г 2,4 г 4,5..."


In [ ]:
### TO-DO train dataset

In [ ]:
num_users = len(user)
num_items = len(items)
data = []
row = []
col = []
#TO-DO
#with open("../input/train.json/train.json") as inf:
#    for i, line in enumerate(inf):
#        j = json.loads(line)
#        for item, rating in j["trainRatings"].items():
#            data.append((-1) ** (int(rating) + 1))
#            row.append(i)
#            col.append(int(item))
train_int = scipy.sparse.coo_matrix((data, (row, col)))
print("created train interactions")
del data, row, col
vect_content = TfidfVectorizer(min_df=90, max_df=0.01, lowercase=False)
tfidf_content = vect_content.fit_transform(items.description)
print("transformed description")
vect_title = TfidfVectorizer(min_df=90, max_df=0.01, lowercase=False)
tfidf_title = vect_title.fit_transform(items.label)
print("transformed label")
identity_items = scipy.sparse.eye(num_items)
item_features = scipy.sparse.hstack(
    [identity_items, tfidf_content, tfidf_title], format="csr"
)
model = lightfm.LightFM(no_components=128, loss="logistic", random_state=0)
print("start training")
model.fit(train_int, epochs=7, num_threads=cpu_count(), item_features=item_features)
print("end training")
sample = pd.read_csv('./kaggle/bench.csv')
sample["Predicted"] = model.predict(
    sample.Id.values,
    sample.Prediction.values,
    item_features=item_features,
    num_threads=cpu_count(),
)
sample = sample.sort_values(by=["Id", "Predicted"], ascending=True)
sample.to_csv("submission.csv", index=False)